<a href="https://colab.research.google.com/github/aseprite1/BOJ_step/blob/main/easy_boj_formatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

초기 설정


In [ ]:
!pip install ipynb-py-convert
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:

import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()

options.add_argument('--headless')        # cHead-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options=options)

print("-----CHOOSE LEVEL-----","info : acmicpc.net/step","Choose level",sep='\n')

level=input()
try:
  if int(level)>54:
    exit(print("WRONG LEVEL!!"))
except:
  print("Please enter a number")
  level=input()

url = "https://www.acmicpc.net/step"
driver.get(url)

# 단계별 사이트 접속
site_url = "/html/body/div[2]/div[2]/div[5]/div/div/table/tbody/tr["+ level +"]/td[2]/a"
site=driver.find_element(By.XPATH, site_url) 
level_name = site.text
site.send_keys('\n')


step_url = driver.current_url
problem_list=[]
table = driver.find_element(By.ID, "problemset")
tbody =table.find_element(By.TAG_NAME, "tbody")
rows = tbody.find_elements(By.TAG_NAME, "tr")

# 문제의 번호, 제목 가져오기
for idx in range(0,len(rows),2):
  problem=(rows[idx].find_elements(By.TAG_NAME, "td")[1].text,rows[idx].find_elements(By.TAG_NAME, "td")[2].text)
  s="### [" + problem[0] + "](https://www.acmicpc.net/problem/"  +  problem[0] +  ") "  +  problem[1]
  problem_list.append("# %%\n'''\n" + s + "\n" + "'''\n\n# %%\n" + "#PASTE YOUR CODE HERE" + "\n\n")

if len(level)==1:
  level="0"+level

# 머릿말 설정
text_py="# %%\n'''\n# **[" + level + "](" + step_url + ")** :: " + level_name + "\n'''\n\n"
text_py = text_py + "".join(problem_list)

# py 파일생성
file_name = level + "_" + level_name+".py"
f=open(file_name, 'w')
f.write(text_py)
f.close()
file_name=file_name[:-3]
print("NEW FILE : \""+file_name+".ipynb\"")
os.environ['FILE_NAME'] = file_name

# py -> ipynb 변환 후 py 삭제
!ipynb-py-convert "$FILE_NAME.py" "$FILE_NAME.ipynb"
!rm "$FILE_NAME.py"

driver.close()

# #colab 사용 시 우측 폴더 아이콘  content에서 결과물 다운로드하면 됨